In [13]:
import numpy as np
import pandas as pd
import datetime
import glob
from pathlib import Path
from scipy import stats
import json

# Login

In [14]:
login = pd.read_csv('../Dataset/login.csv', index_col='login')
login

,country,account_currency,reg_date
login,,,
457547,Romania,EUR,1614212132
474589,CA,CAD,1609987442
504321,CA,CAD,1602642710
504322,CA,USD,1602736545
504326,CA,USD,1603093152
...,...,...,...
1000055123,Canada,USD,1699344255
1000055441,Korea,USD,1702437127
1000055515,Netherlands,EUR,1703243841


In [15]:
login['Datetime'] = pd.to_datetime(login['reg_date'],unit='s')
login

,country,account_currency,reg_date,Datetime
login,,,,
457547,Romania,EUR,1614212132,2021-02-25 00:15:32
474589,CA,CAD,1609987442,2021-01-07 02:44:02
504321,CA,CAD,1602642710,2020-10-14 02:31:50
504322,CA,USD,1602736545,2020-10-15 04:35:45
504326,CA,USD,1603093152,2020-10-19 07:39:12
...,...,...,...,...
1000055123,Canada,USD,1699344255,2023-11-07 08:04:15
1000055441,Korea,USD,1702437127,2023-12-13 03:12:07
1000055515,Netherlands,EUR,1703243841,2023-12-22 11:17:21


In [16]:
# Read the country mapping from the JSON file
with open("../country_mapping.json", "r") as json_file:
    country_mapping = json.load(json_file)

In [17]:
login['country'] = login['country'].replace(country_mapping)
login

,country,account_currency,reg_date,Datetime
login,,,,
457547,Romania,EUR,1614212132,2021-02-25 00:15:32
474589,Canada,CAD,1609987442,2021-01-07 02:44:02
504321,Canada,CAD,1602642710,2020-10-14 02:31:50
504322,Canada,USD,1602736545,2020-10-15 04:35:45
504326,Canada,USD,1603093152,2020-10-19 07:39:12
...,...,...,...,...
1000055123,Canada,USD,1699344255,2023-11-07 08:04:15
1000055441,South Korea,USD,1702437127,2023-12-13 03:12:07
1000055515,Netherlands,EUR,1703243841,2023-12-22 11:17:21


# Trades

In [18]:
trades = pd.read_csv('../Dataset/trades.csv', index_col='ticket')
trades

,login,symbol,cmd,volume,open_time,open_price,close_time,close_price,tp,sl,reason,commission,swaps,profit,volume_usd
ticket,,,,,,,,,,,,,,,
68880703,7062462,XAUUSD,0,0.01,1707843941,1991.35000,1708013114,2003.01000,0.00000,0.00000,1,0.00,-1.12,10.83,3994.36
68880910,7062462,XAUUSD,0,0.02,1707844129,1990.30000,1708013110,2002.76000,0.00000,0.00000,1,0.00,-2.25,23.16,7986.12
68120690,813125,US2000,0,1.00,1706083005,1983.22000,1706114503,1996.68000,2012.34000,1980.03000,1,0.00,0.00,105.42,39799.00
68169249,813125,US2000,0,2.00,1706139371,1960.62000,1706200143,1991.51000,2331.85000,1958.94000,1,0.00,-7.21,485.26,79042.60
68186877,88945036,NZDUSD,0,0.13,1706169600,0.61062,1706197106,0.61317,0.61312,0.55059,1,-0.91,0.00,33.15,15909.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68795989,810950,XAUUSD,0,0.10,1707738783,2020.74000,1707810805,2021.51000,2026.00000,0.00000,5,0.00,0.00,7.70,40422.50
68796026,810950,XAUUSD,0,0.10,1707738794,2020.70000,1707810809,2021.50000,2026.00000,0.00000,5,0.00,0.00,8.00,40422.00
68854039,810799,EURUSD,1,0.10,1707825414,1.07654,1707825616,1.07693,0.00000,1.07694,5,-0.60,0.00,-3.62,21534.70


In [19]:
trades['Open_Datetime'] = pd.to_datetime(trades['open_time'],unit='s')
trades['Close_Datetime'] = pd.to_datetime(trades['close_time'],unit='s')
trades

,login,symbol,cmd,volume,open_time,open_price,close_time,close_price,tp,sl,reason,commission,swaps,profit,volume_usd,Open_Datetime,Close_Datetime
ticket,,,,,,,,,,,,,,,,,
68880703,7062462,XAUUSD,0,0.01,1707843941,1991.35000,1708013114,2003.01000,0.00000,0.00000,1,0.00,-1.12,10.83,3994.36,2024-02-13 17:05:41,2024-02-15 16:05:14
68880910,7062462,XAUUSD,0,0.02,1707844129,1990.30000,1708013110,2002.76000,0.00000,0.00000,1,0.00,-2.25,23.16,7986.12,2024-02-13 17:08:49,2024-02-15 16:05:10
68120690,813125,US2000,0,1.00,1706083005,1983.22000,1706114503,1996.68000,2012.34000,1980.03000,1,0.00,0.00,105.42,39799.00,2024-01-24 07:56:45,2024-01-24 16:41:43
68169249,813125,US2000,0,2.00,1706139371,1960.62000,1706200143,1991.51000,2331.85000,1958.94000,1,0.00,-7.21,485.26,79042.60,2024-01-24 23:36:11,2024-01-25 16:29:03
68186877,88945036,NZDUSD,0,0.13,1706169600,0.61062,1706197106,0.61317,0.61312,0.55059,1,-0.91,0.00,33.15,15909.27,2024-01-25 08:00:00,2024-01-25 15:38:26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68795989,810950,XAUUSD,0,0.10,1707738783,2020.74000,1707810805,2021.51000,2026.00000,0.00000,5,0.00,0.00,7.70,40422.50,2024-02-12 11:53:03,2024-02-13 07:53:25
68796026,810950,XAUUSD,0,0.10,1707738794,2020.70000,1707810809,2021.50000,2026.00000,0.00000,5,0.00,0.00,8.00,40422.00,2024-02-12 11:53:14,2024-02-13 07:53:29
68854039,810799,EURUSD,1,0.10,1707825414,1.07654,1707825616,1.07693,0.00000,1.07694,5,-0.60,0.00,-3.62,21534.70,2024-02-13 11:56:54,2024-02-13 12:00:16


In [20]:
total_com = trades[['login', 'commission']].groupby('login').sum()
total_com

,commission
login,
504326,0.00
504336,0.00
504357,-0.14
504359,0.00
504361,0.00
...,...
88945036,-3619.00
88945038,-1236.72
1000054825,0.00


In [21]:
total_com_sort = total_com.sort_values(by=['commission']).where(total_com['commission'] != 0).dropna()
total_com_sort

,commission
login,
822580,-34917.28
524760,-19583.40
507711,-17752.63
770911,-15251.57
813375,-13835.13
...,...
7063895,-0.06
88944293,-0.06
508682,-0.06


In [22]:
trades[trades['login'] == 7059146]

,login,symbol,cmd,volume,open_time,open_price,close_time,close_price,tp,sl,reason,commission,swaps,profit,volume_usd,Open_Datetime,Close_Datetime
ticket,,,,,,,,,,,,,,,,,
68575202,7059146,EURUSD,0,3.48,1707206401,1.07535,1707210451,1.07599,0.0,0.0,1,-26.10,0.00,206.99,7.486663e+05,2024-02-06 08:00:01,2024-02-06 09:07:31
68575534,7059146,USDJPY,1,1.28,1707207300,148.39500,1707239439,148.33500,0.0,0.0,1,-9.60,0.00,48.20,5.618822e+09,2024-02-06 08:15:00,2024-02-06 17:10:39
68578926,7059146,EURCHF,0,1.71,1707214501,0.93635,1707215424,0.93674,0.0,0.0,1,-12.83,0.00,71.19,2.785731e+05,2024-02-06 10:15:01,2024-02-06 10:30:24
68578934,7059146,EURJPY,0,1.04,1707214501,159.81900,1707315043,159.57100,0.0,0.0,1,-7.80,7.35,-161.63,4.917958e+09,2024-02-06 10:15:01,2024-02-07 14:10:43
68578943,7059146,GBPCHF,0,1.16,1707214502,1.09323,1707215424,1.09369,0.0,0.0,1,-8.70,0.00,56.96,2.206355e+05,2024-02-06 10:15:02,2024-02-06 10:30:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68860563,7059146,EURUSD,0,1.40,1707831000,1.07811,1707836456,1.07866,0.0,0.0,1,-10.50,0.00,71.38,3.019478e+05,2024-02-13 13:30:00,2024-02-13 15:00:56
68862987,7059146,EURCAD,0,1.63,1707833701,1.44996,1707836523,1.45058,0.0,0.0,1,-12.23,0.00,69.67,6.412566e+05,2024-02-13 14:15:01,2024-02-13 15:02:03
68787228,7059146,CHFJPY,0,2.08,1707732011,170.82700,1707816019,170.87000,0.0,0.0,1,-15.60,5.32,55.47,1.066486e+10,2024-02-12 10:00:11,2024-02-13 09:20:19


In [23]:
login[login.index.isin([7059146, 7059314, 822580, 7059153, 7059142, 7054378, 806283, 7054778, 7057660, 7064409])]

,country,account_currency,reg_date,Datetime
login,,,,
806283,Singapore,SGD,1676428199,2023-02-15 02:29:59
822580,Austria,CHF,1695714771,2023-09-26 07:52:51
7054378,Belgium,EUR,1697819165,2023-10-20 16:26:05
7054778,Turkey,USD,1698180721,2023-10-24 20:52:01
7057660,Austria,USD,1700156816,2023-11-16 17:46:56
7059142,Austria,EUR,1701165234,2023-11-28 09:53:54
7059146,Austria,EUR,1701165611,2023-11-28 10:00:11
7059153,Austria,EUR,1701166587,2023-11-28 10:16:27
7059314,Austria,CHF,1701262490,2023-11-29 12:54:50


# Daily Report

In [25]:
report = pd.read_csv('../Dataset/daily_report.csv')
report

,login,record_time,net_deposit,balance,equity,credit,profit_closed,profit_floating,margin
0,457547,2023-01-01,0.0,0.00,0.00,0.0,0.00,0.00,0.00
1,474589,2023-01-01,0.0,0.56,0.56,0.0,0.00,0.00,0.00
2,504321,2023-01-01,0.0,2.03,2.03,0.0,0.00,0.00,0.00
3,504322,2023-01-01,0.0,0.51,0.51,0.0,0.00,0.00,0.00
4,504326,2023-01-01,0.0,0.01,0.01,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...
8664156,1000055123,2024-02-29,0.0,37521.00,36507.56,0.0,-122.18,-1013.44,818.92
8664157,1000055441,2024-02-29,0.0,0.00,0.00,0.0,0.00,0.00,0.00
8664158,1000055515,2024-02-29,0.0,0.00,0.00,0.0,0.00,0.00,0.00
8664159,1000059485,2024-02-29,0.0,0.00,0.00,0.0,0.00,0.00,0.00


In [26]:
sorted_report = report.sort_values('record_time')
sorted_report

,login,record_time,net_deposit,balance,equity,credit,profit_closed,profit_floating,margin
0,457547,2023-01-01,0.0,0.00,0.00,0.0,0.0,0.0,0.0
8499,523653,2023-01-01,0.0,82.16,82.16,0.0,0.0,0.0,0.0
8500,523655,2023-01-01,0.0,0.00,0.00,0.0,0.0,0.0,0.0
8501,523659,2023-01-01,0.0,5724.18,5724.18,0.0,0.0,0.0,0.0
8502,523660,2023-01-01,0.0,0.00,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
8644286,780550,2024-02-29,0.0,0.77,0.77,0.0,0.0,0.0,0.0
8644285,780549,2024-02-29,0.0,0.31,0.31,0.0,0.0,0.0,0.0
8644284,780546,2024-02-29,0.0,0.28,0.28,0.0,0.0,0.0,0.0
8644296,780575,2024-02-29,0.0,5.99,5.99,0.0,0.0,0.0,0.0


# Daily Chart

In [27]:
daily_charts = glob.glob("../Dataset/daily_chart/*.csv")
all_charts = pd.DataFrame()

csv = pd.read_csv('../Dataset/daily_chart/AUDUSD.csv', index_col='date', parse_dates=True)
csv

,close
date,
2023-01-03,0.67233
2023-01-04,0.68296
2023-01-05,0.67513
2023-01-06,0.68674
2023-01-09,0.69091
...,...
2024-02-23,0.65621
2024-02-26,0.65397
2024-02-27,0.65426


In [28]:
daily_charts = glob.glob("../Dataset/daily_chart/*.csv")
all_charts = pd.DataFrame()

for chart in daily_charts:
    name = Path(chart).stem
    csv = pd.read_csv(chart, index_col='date', parse_dates=True)
    all_charts[name] = csv['close']
    
all_charts

,AUDUSD,EURUSD,GBPUSD,NZDUSD,USDCAD,USDCHF,USDCNH,USDHKD,USDHUF,USDJPY,USDMXN,USDNOK,USDPLN,USDSEK,USDSGD,USDTHB,USDTRY,USDZAR
date,,,,,,,,,,,,,,,,,,
2023-01-03,0.67233,1.05472,1.19663,0.62510,1.36606,0.93562,6.92264,7.81234,379.240,131.006,19.40581,10.07969,4.43075,10.56455,1.34486,34.36595,18.69119,17.00699
2023-01-04,0.68296,1.06030,1.20527,0.62923,1.34773,0.92902,6.89772,7.81581,372.910,132.613,19.36711,10.07639,4.39765,10.50299,1.33905,33.90395,18.72579,16.87099
2023-01-05,0.67513,1.05207,1.18981,0.62292,1.35678,0.93626,6.88809,7.81272,376.260,133.397,19.32226,10.24325,4.45195,10.66025,1.34521,34.02995,18.75219,17.16219
2023-01-06,0.68674,1.06437,1.20924,0.63485,1.34434,0.92651,6.82917,7.80552,370.090,132.044,19.12619,9.99835,4.40495,10.50815,1.33458,33.68795,18.73192,17.08855
2023-01-09,0.69091,1.07288,1.21725,0.63708,1.33867,0.92091,6.77883,7.80290,369.190,131.882,19.13564,9.91625,4.37275,10.38285,1.33031,33.39095,18.73628,16.94529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-23,0.65621,1.08190,1.26710,0.61912,1.34960,0.88107,7.19926,7.81768,353.709,150.512,17.09442,10.53685,NaN,10.31995,1.34089,35.93995,30.35980,19.27635
2024-02-26,0.65397,1.08503,1.26843,0.61720,1.35043,0.87993,7.20879,7.82367,358.570,150.693,17.09630,10.51215,NaN,10.28105,1.34432,35.87695,31.07723,19.28385
2024-02-27,0.65426,1.08435,1.26837,0.61698,1.35270,0.87853,7.21219,7.82395,360.300,150.501,17.06220,10.52735,NaN,10.30345,1.34387,35.86595,31.14562,19.08745


# Symbol

In [29]:
symbol = pd.read_csv('../Dataset/symbol.csv', index_col='symbol')
symbol

,description,type
symbol,,
ALUMINIUM,Aluminium vs US Dollar,Commodity
COPPER,Copper vs US Dollar,Commodity
GAS,Natural Gas vs US Dollar,Commodity
LEAD,Lead vs US Dollar,Commodity
NICKEL,Nickel vs US Dollar,Commodity
...,...,...
XAUUSD,Gold vs US Dollar,Metal
XPDUSD,Palladium vs US Dollar,Metal
XPTUSD,Platinum vs US Dollar,Metal


# Reason

In [30]:
reason = pd.read_csv('../Dataset/reason.csv', index_col='code')
reason

,reason
code,
0,Client
1,Expert
2,Dealer
3,Signal
4,Gateway
5,Mobile
6,Web
7,API


# Combined

In [31]:
login_trades = trades.join(login, how='inner', on='login').set_index(['login', trades.index])
login_trades

symbol  cmd  volume   open_time   open_price  close_time  \
login   ticket                                                               
7062462 68880703  XAUUSD    0    0.01  1707843941   1991.35000  1708013114   
        68880910  XAUUSD    0    0.02  1707844129   1990.30000  1708013110   
        68120690  XAUUSD    0    0.02  1707843610   1995.38000  1708008593   
        68169249  XAUUSD    0    0.01  1707840008   2003.21000  1708013129   
        68186877    DJ30    1    0.01  1707994496  38495.30000  1707994498   
...                  ...  ...     ...         ...          ...         ...   
814228  68795989  NAS100    1    0.05  1709226605  18008.40000  1709226626   
        68796026  NAS100    1    0.05  1709228572  17986.35000  1709228606   
        68854039  NAS100    1    0.05  1709241700  18015.35000  1709241909   
771049  68785760  AUDUSD    1    0.01  1707719895      0.65189  1707766172   
        68816189  AUDUSD    1    0.20  1707720344      0.65169  1707766172   

                  close_price           tp           sl  reason  commission  \
login   ticket                                                                
7062462 68880703   2003.01000      0.00000      0.00000       1         0.0   
        68880910   2002.76000      0.00000      0.00000       1         0.0   
        68120690   1998.78000      0.00000      0.00000       1         0.0   
        68169249   2002.88000      0.00000      0.00000       1         0.0   
        68186877  38496.60000      0.00000      0.00000       1         0.0   
...                       ...          ...          ...     ...         ...   
814228  68795989  17993.75000      0.00000      0.00000       5         0.0   
        68796026  17976.50000  17976.00000  17996.00000       5         0.0   
        68854039  18004.92000  18005.00000  18025.00000       5         0.0   
771049  68785760      0.65401      0.64490      0.65402       0         0.0   
        68816189      0.65401      0.64492      0.65402       0         0.0   

                  swaps  profit  volume_usd       Open_Datetime  \
login   ticket                                                    
7062462 68880703  -1.12   10.83     3994.36 2024-02-13 17:05:41   
        68880910  -2.25   23.16     7986.12 2024-02-13 17:08:49   
        68120690  -2.25    6.33     7988.32 2024-02-13 17:00:10   
        68169249  -1.12   -0.30     4006.09 2024-02-13 16:00:08   
        68186877   0.00   -0.12     7699.19 2024-02-15 10:54:56   
...                 ...     ...         ...                 ...   
814228  68795989   0.00    7.33    18001.08 2024-02-29 17:10:05   
        68796026   0.00    4.93    17981.42 2024-02-29 17:42:52   
        68854039   0.00    5.22    18010.13 2024-02-29 21:21:40   
771049  68785760   0.00   -2.12     1305.90 2024-02-12 06:38:15   
        68816189   0.00  -46.40    26114.00 2024-02-12 06:45:44   

                      Close_Datetime country account_currency    reg_date  \
login   ticket                                                              
7062462 68880703 2024-02-15 16:05:14  Latvia              EUR  1703271119   
        68880910 2024-02-15 16:05:10  Latvia              EUR  1703271119   
        68120690 2024-02-15 14:49:53  Latvia              EUR  1703271119   
        68169249 2024-02-15 16:05:29  Latvia              EUR  1703271119   
        68186877 2024-02-15 10:54:58  Latvia              EUR  1703271119   
...                              ...     ...              ...         ...   
814228  68795989 2024-02-29 17:10:26  Canada              USD  1685994075   
        68796026 2024-02-29 17:43:26  Canada              USD  1685994075   
        68854039 2024-02-29 21:25:09  Canada              USD  1685994075   
771049  68785760 2024-02-12 19:29:32   Ghana              USD  1561018542   
        68816189 2024-02-12 19:29:32   Ghana              USD  1561018542   

                            Datetime  
login   ticket                        
7062462 68880703 2023-1

In [32]:
test_subj = login_trades.loc[[7062462]]
test_subj

symbol  cmd  volume   open_time   open_price  close_time  \
login   ticket                                                               
7062462 68880703  XAUUSD    0    0.01  1707843941   1991.35000  1708013114   
        68880910  XAUUSD    0    0.02  1707844129   1990.30000  1708013110   
        68120690  XAUUSD    0    0.02  1707843610   1995.38000  1708008593   
        68169249  XAUUSD    0    0.01  1707840008   2003.21000  1708013129   
        68186877    DJ30    1    0.01  1707994496  38495.30000  1707994498   
...                  ...  ...     ...         ...          ...         ...   
        68314931  GBPUSD    1    0.01  1707756900      1.26242  1707813584   
        68223546  EURUSD    1    0.02  1707815102      1.07730  1707817261   
        68402535    DJ30    1    0.01  1707816717  38828.30000  1707816745   
        66767288    DJ30    1    0.01  1707816903  38831.30000  1707817282   
        66767310    DJ30    1    0.01  1707817457  38830.30000  1707827358   

                  close_price   tp   sl  reason  commission  swaps  profit  \
login   ticket                                                               
7062462 68880703   2003.01000  0.0  0.0       1         0.0  -1.12   10.83   
        68880910   2002.76000  0.0  0.0       1         0.0  -2.25   23.16   
        68120690   1998.78000  0.0  0.0       1         0.0  -2.25    6.33   
        68169249   2002.88000  0.0  0.0       1         0.0  -1.12   -0.30   
        68186877  38496.60000  0.0  0.0       1         0.0   0.00   -0.12   
...                       ...  ...  ...     ...         ...    ...     ...   
        68314931      1.26210  0.0  0.0       1         0.0   0.00    0.29   
        68223546      1.07682  0.0  0.0       1         0.0   0.00    0.89   
        68402535  38827.60000  0.0  0.0       1         0.0   0.00    0.06   
        66767288  38829.60000  0.0  0.0       1         0.0   0.00    0.16   
        66767310  38779.10000  0.0  0.0       1         0.0   0.00    4.75   

                  volume_usd       Open_Datetime      Close_Datetime country  \
login   ticket                                                                 
7062462 68880703     3994.36 2024-02-13 17:05:41 2024-02-15 16:05:14  Latvia   
        68880910     7986.12 2024-02-13 17:08:49 2024-02-15 16:05:10  Latvia   
        68120690     7988.32 2024-02-13 17:00:10 2024-02-15 14:49:53  Latvia   
        68169249     4006.09 2024-02-13 16:00:08 2024-02-15 16:05:29  Latvia   
        68186877     7699.19 2024-02-15 10:54:56 2024-02-15 10:54:58  Latvia   
...                      ...                 ...                 ...     ...   
        68314931     2524.52 2024-02-12 16:55:00 2024-02-13 08:39:44  Latvia   
        68223546     4308.24 2024-02-13 09:05:02 2024-02-13 09:41:01  Latvia   
        68402535     7765.59 2024-02-13 09:31:57 2024-02-13 09:32:25  Latvia   
        66767288     7766.09 2024-02-13 09:35:03 2024-02-13 09:41:22  Latvia   
        66767310     7760.94 2024-02-13 09:44:17 2024-02-13 12:29:18  Latvia   

                 account_currency    reg_date            Datetime  
login   ticket                                                     
7062462 68880703              EUR  1703271119 2023-12-22 18:51:59  
        68880910              EUR  1703271119 2023-12-22 18:51:59  
        68120690              EUR  1703271119 2023-12-22 18:51:59  
        68169249              EUR  1703271119 2023-12-22 18:51:59  
        68186877              EUR  1703271119 2023-12-22 18:51:59  
...                           ...         ...                 ...  
        68314931              EUR  1703271119 2023-12-22 18:51:59  
        68223546              EUR  1703271119 2023-12-22 18:51:59  
        68402535              EUR  1703271119 2023-12-22 18:51:59  
        66767288              EUR  1703271119 2023-12-22 18:51:59  
        66767310              EUR  1703271119 2023-12-22 18:51:59  

[1363 rows x 20 columns]

In [33]:
earliest_trade_open = test_subj['Open_Datetime'].min()
latest_trade_close = test_subj['Close_Datetime'].max()
print('Earliest Date: ', earliest_trade_open)
print('Latest Date: ', latest_trade_close)
longevity = latest_trade_close - earliest_trade_open
print('Longevity: ', longevity)

Earliest Date:  2024-01-02 01:25:21
Latest Date:  2024-02-29 18:23:46
Longevity:  58 days 16:58:25


In [34]:
login_trades['Open_Datetime'].groupby(level=0).min()

login
504326       2023-05-10 20:32:01
504336       2023-05-18 09:39:15
504357       2023-09-05 16:33:46
504359       2023-05-12 10:47:46
504361       2023-10-24 20:14:33
                     ...        
88945036     2023-06-07 11:19:09
88945038     2023-05-18 04:45:01
1000054825   2023-10-03 09:10:43
1000054838   2023-10-23 21:25:00
1000055123   2023-11-13 01:39:55
Name: Open_Datetime, Length: 11976, dtype: datetime64[ns]

In [35]:
login_trades.loc[[504326]]['Open_Datetime'].min()

Timestamp('2023-05-10 20:32:01')

In [36]:
earliest_trades_opened = login_trades['Open_Datetime'].groupby(level=0).min()
latest_trades_closed = login_trades['Close_Datetime'].groupby(level=0).max()
first_and_last_trades = earliest_trades_opened.to_frame().join(latest_trades_closed, how='inner')
first_and_last_trades['fl_trade_gap'] = first_and_last_trades['Close_Datetime'] - first_and_last_trades['Open_Datetime']
first_and_last_trades

,Open_Datetime,Close_Datetime,fl_trade_gap
login,,,
504326,2023-05-10 20:32:01,2024-02-29 23:02:04,295 days 02:30:03
504336,2023-05-18 09:39:15,2023-10-12 19:55:46,147 days 10:16:31
504357,2023-09-05 16:33:46,2023-09-05 17:15:46,0 days 00:42:00
504359,2023-05-12 10:47:46,2024-02-29 17:32:58,293 days 06:45:12
504361,2023-10-24 20:14:33,2023-10-31 17:00:26,6 days 20:45:53
...,...,...,...
88945036,2023-06-07 11:19:09,2024-02-29 19:31:49,267 days 08:12:40
88945038,2023-05-18 04:45:01,2024-01-26 13:06:27,253 days 08:21:26
1000054825,2023-10-03 09:10:43,2024-02-23 19:39:46,143 days 10:29:03


In [37]:
logins_2023 = login[login['Datetime'] > '2023-01-01']
logins_2023

,country,account_currency,reg_date,Datetime
login,,,,
524972,France,EUR,1683610004,2023-05-09 05:26:44
524974,Switzerland,USD,1684293182,2023-05-17 03:13:02
524976,Austria,EUR,1686117078,2023-06-07 05:51:18
524978,Austria,EUR,1686117516,2023-06-07 05:58:36
524979,France,USD,1686118650,2023-06-07 06:17:30
...,...,...,...,...
1000055123,Canada,USD,1699344255,2023-11-07 08:04:15
1000055441,South Korea,USD,1702437127,2023-12-13 03:12:07
1000055515,Netherlands,EUR,1703243841,2023-12-22 11:17:21


To summarise, in addition to the normal login info, the df below shows when the account's first trade was opened, when the last trade was closed, the gap between the two, and the gap between the account being created and the first trade being opened for accounts that were opened after the start of 2023.


In [38]:
login_fl_trades = logins_2023.join(first_and_last_trades, how='inner')
login_fl_trades['rf_trade_gap'] = login_fl_trades['Open_Datetime'] - login_fl_trades['Datetime']  
login_fl_trades

,country,account_currency,reg_date,Datetime,Open_Datetime,Close_Datetime,fl_trade_gap,rf_trade_gap
login,,,,,,,,
524974,Switzerland,USD,1684293182,2023-05-17 03:13:02,2023-05-19 06:00:13,2024-02-29 22:02:00,286 days 16:01:47,2 days 02:47:11
524978,Austria,EUR,1686117516,2023-06-07 05:58:36,2023-07-11 21:56:32,2023-12-01 20:54:51,142 days 22:58:19,34 days 15:57:56
524979,France,USD,1686118650,2023-06-07 06:17:30,2023-06-16 17:04:15,2023-10-02 08:42:48,107 days 15:38:33,9 days 10:46:45
524984,Singapore,USD,1698741248,2023-10-31 08:34:08,2023-11-13 08:54:56,2024-02-13 15:30:12,92 days 06:35:16,13 days 00:20:48
760487,Singapore,SGD,1672822104,2023-01-04 08:48:24,2023-01-05 16:50:56,2023-01-18 21:11:30,13 days 04:20:34,1 days 08:02:32
...,...,...,...,...,...,...,...,...
88945036,Nigeria,USD,1682915786,2023-05-01 04:36:26,2023-06-07 11:19:09,2024-02-29 19:31:49,267 days 08:12:40,37 days 06:42:43
88945038,Singapore,SGD,1683285920,2023-05-05 11:25:20,2023-05-18 04:45:01,2024-01-26 13:06:27,253 days 08:21:26,12 days 17:19:41
1000054825,Singapore,SGD,1695644841,2023-09-25 12:27:21,2023-10-03 09:10:43,2024-02-23 19:39:46,143 days 10:29:03,7 days 20:43:22


In [39]:
login_fl_trades['trades'] = login_trades['cmd'].groupby(level=0).count()
login_fl_trades['daily_trades'] = login_fl_trades['trades'] / login_fl_trades['fl_trade_gap'].dt.days
login_fl_trades

,country,account_currency,reg_date,Datetime,Open_Datetime,Close_Datetime,fl_trade_gap,rf_trade_gap,trades,daily_trades
login,,,,,,,,,,
524974,Switzerland,USD,1684293182,2023-05-17 03:13:02,2023-05-19 06:00:13,2024-02-29 22:02:00,286 days 16:01:47,2 days 02:47:11,143,0.500000
524978,Austria,EUR,1686117516,2023-06-07 05:58:36,2023-07-11 21:56:32,2023-12-01 20:54:51,142 days 22:58:19,34 days 15:57:56,1392,9.802817
524979,France,USD,1686118650,2023-06-07 06:17:30,2023-06-16 17:04:15,2023-10-02 08:42:48,107 days 15:38:33,9 days 10:46:45,2194,20.504673
524984,Singapore,USD,1698741248,2023-10-31 08:34:08,2023-11-13 08:54:56,2024-02-13 15:30:12,92 days 06:35:16,13 days 00:20:48,244,2.652174
760487,Singapore,SGD,1672822104,2023-01-04 08:48:24,2023-01-05 16:50:56,2023-01-18 21:11:30,13 days 04:20:34,1 days 08:02:32,69,5.307692
...,...,...,...,...,...,...,...,...,...,...
88945036,Nigeria,USD,1682915786,2023-05-01 04:36:26,2023-06-07 11:19:09,2024-02-29 19:31:49,267 days 08:12:40,37 days 06:42:43,3555,13.314607
88945038,Singapore,SGD,1683285920,2023-05-05 11:25:20,2023-05-18 04:45:01,2024-01-26 13:06:27,253 days 08:21:26,12 days 17:19:41,1306,5.162055
1000054825,Singapore,SGD,1695644841,2023-09-25 12:27:21,2023-10-03 09:10:43,2024-02-23 19:39:46,143 days 10:29:03,7 days 20:43:22,829,5.797203


In [40]:
login_report = pd.merge(logins_2023, report, how='inner', on='login').set_index(['login', 'record_time'])
login_report

country account_currency    reg_date  \
login      record_time                                        
524972     2023-05-09   France              EUR  1683610004   
           2023-05-10   France              EUR  1683610004   
           2023-05-11   France              EUR  1683610004   
           2023-05-12   France              EUR  1683610004   
           2023-05-13   France              EUR  1683610004   
...                        ...              ...         ...   
1000059496 2024-02-24   Canada              USD  1705579537   
           2024-02-25   Canada              USD  1705579537   
           2024-02-26   Canada              USD  1705579537   
           2024-02-27   Canada              USD  1705579537   
           2024-02-29   Canada              USD  1705579537   

                                  Datetime  net_deposit  balance  equity  \
login      record_time                                                     
524972     2023-05-09  2023-05-09 05:26:44          0.0      0.0     0.0   
           2023-05-10  2023-05-09 05:26:44          0.0      0.0     0.0   
           2023-05-11  2023-05-09 05:26:44          0.0      0.0     0.0   
           2023-05-12  2023-05-09 05:26:44          0.0      0.0     0.0   
           2023-05-13  2023-05-09 05:26:44          0.0      0.0     0.0   
...                                    ...          ...      ...     ...   
1000059496 2024-02-24  2024-01-18 12:05:37          0.0      0.0     0.0   
           2024-02-25  2024-01-18 12:05:37          0.0      0.0     0.0   
           2024-02-26  2024-01-18 12:05:37          0.0      0.0     0.0   
           2024-02-27  2024-01-18 12:05:37          0.0      0.0     0.0   
           2024-02-29  2024-01-18 12:05:37          0.0      0.0     0.0   

                        credit  profit_closed  profit_floating  margin  
login      record_time                                                  
524972     2023-05-09      0.0            0.0              0.0     0.0  
           2023-05-10      0.0            0.0              0.0     0.0  
           2023-05-11      0.0            0.0              0.0     0.0  
           2023-05-12      0.0            0.0              0.0     0.0  
           2023-05-13      0.0            0.0              0.0     0.0  
...                        ...            ...              ...     ...  
1000059496 2024-02-24      0.0            0.0              0.0     0.0  
           2024-02-25      0.0            0.0              0.0     0.0  
           2024-02-26      0.0            0.0              0.0     0.0  
           2024-02-27      0.0            0.0              0.0     0.0  
           2024-02-29      0.0            0.0              0.0     0.0  

[3661434 rows x 11 columns]

SEE BELOW FOR CURRENCY CONVERSION SHENANIGANS

In [41]:
usd_columns = [col for col in all_charts.columns if col.startswith('USD')]
all_charts_USD = all_charts.copy()
all_charts_USD[usd_columns] = 1 / all_charts_USD[usd_columns]
all_charts_USD.columns = [col.replace('USD', '') for col in all_charts_USD.columns]
all_charts_USD

,AUD,EUR,GBP,NZD,CAD,CHF,CNH,HKD,HUF,JPY,MXN,NOK,PLN,SEK,SGD,THB,TRY,ZAR
date,,,,,,,,,,,,,,,,,,
2023-01-03,0.67233,1.05472,1.19663,0.62510,0.732032,1.068810,0.144454,0.128003,0.002637,0.007633,0.051531,0.099209,0.225695,0.094656,0.743572,0.029099,0.053501,0.058799
2023-01-04,0.68296,1.06030,1.20527,0.62923,0.741988,1.076403,0.144975,0.127946,0.002682,0.007541,0.051634,0.099242,0.227394,0.095211,0.746798,0.029495,0.053402,0.059273
2023-01-05,0.67513,1.05207,1.18981,0.62292,0.737039,1.068079,0.145178,0.127996,0.002658,0.007496,0.051754,0.097625,0.224621,0.093806,0.743378,0.029386,0.053327,0.058268
2023-01-06,0.68674,1.06437,1.20924,0.63485,0.743859,1.079319,0.146431,0.128114,0.002702,0.007573,0.052284,0.100017,0.227017,0.095164,0.749299,0.029684,0.053385,0.058519
2023-01-09,0.69091,1.07288,1.21725,0.63708,0.747010,1.085882,0.147518,0.128157,0.002709,0.007583,0.052259,0.100845,0.228689,0.096313,0.751704,0.029948,0.053372,0.059013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-23,0.65621,1.08190,1.26710,0.61912,0.740960,1.134984,0.138903,0.127915,0.002827,0.006644,0.058499,0.094905,NaN,0.096900,0.745773,0.027824,0.032938,0.051877
2024-02-26,0.65397,1.08503,1.26843,0.61720,0.740505,1.136454,0.138720,0.127817,0.002789,0.006636,0.058492,0.095128,NaN,0.097266,0.743871,0.027873,0.032178,0.051857
2024-02-27,0.65426,1.08435,1.26837,0.61698,0.739262,1.138265,0.138654,0.127813,0.002775,0.006644,0.058609,0.094991,NaN,0.097055,0.744120,0.027882,0.032107,0.052390


In [42]:
# Step 1: Identify rows where account_currency is not USD
non_usd_rows = login_report[login_report['account_currency'] != 'USD']

# Step 2: Merge non_usd_rows with all_charts_USD to get conversion rates
merged_data = non_usd_rows.merge(all_charts_USD, how='left', left_on='Datetime', right_index=True)

# Step 3: Multiply values in specified columns with conversion rates
conversion_columns = ['net_deposit', 'balance', 'equity', 'credit', 'profit_closed', 'profit_floating', 'margin']
converted_rows = pd.DataFrame(index=merged_data.index)

# Step 4: Copy over non-converted columns to the login_report_USD DataFrame
non_conversion_columns = [col for col in login_report.columns if col not in conversion_columns]
for column in non_conversion_columns:
    converted_rows[column] = merged_data[column]
for column in conversion_columns:
    converted_rows[column] = merged_data[column.replace('USD', '')] * merged_data[column]

# Step 5: Identify rows where account_currency is USD
usd_rows = login_report[login_report['account_currency'] == 'USD']

# Step 6: Combine converted and usd rows
login_report_USD = pd.concat([converted_rows] + [usd_rows], axis=0)


In [43]:
login_report_USD

country account_currency    reg_date  \
login      record_time                                        
524972     2023-05-09   France              EUR  1683610004   
           2023-05-10   France              EUR  1683610004   
           2023-05-11   France              EUR  1683610004   
           2023-05-12   France              EUR  1683610004   
           2023-05-13   France              EUR  1683610004   
...                        ...              ...         ...   
1000059496 2024-02-24   Canada              USD  1705579537   
           2024-02-25   Canada              USD  1705579537   
           2024-02-26   Canada              USD  1705579537   
           2024-02-27   Canada              USD  1705579537   
           2024-02-29   Canada              USD  1705579537   

                                  Datetime  net_deposit  balance  equity  \
login      record_time                                                     
524972     2023-05-09  2023-05-09 05:26:44          0.0      0.0     0.0   
           2023-05-10  2023-05-09 05:26:44          0.0      0.0     0.0   
           2023-05-11  2023-05-09 05:26:44          0.0      0.0     0.0   
           2023-05-12  2023-05-09 05:26:44          0.0      0.0     0.0   
           2023-05-13  2023-05-09 05:26:44          0.0      0.0     0.0   
...                                    ...          ...      ...     ...   
1000059496 2024-02-24  2024-01-18 12:05:37          0.0      0.0     0.0   
           2024-02-25  2024-01-18 12:05:37          0.0      0.0     0.0   
           2024-02-26  2024-01-18 12:05:37          0.0      0.0     0.0   
           2024-02-27  2024-01-18 12:05:37          0.0      0.0     0.0   
           2024-02-29  2024-01-18 12:05:37          0.0      0.0     0.0   

                        credit  profit_closed  profit_floating  margin  
login      record_time                                                  
524972     2023-05-09      0.0            0.0              0.0     0.0  
           2023-05-10      0.0            0.0              0.0     0.0  
           2023-05-11      0.0            0.0              0.0     0.0  
           2023-05-12      0.0            0.0              0.0     0.0  
           2023-05-13      0.0            0.0              0.0     0.0  
...                        ...            ...              ...     ...  
1000059496 2024-02-24      0.0            0.0              0.0     0.0  
           2024-02-25      0.0            0.0              0.0     0.0  
           2024-02-26      0.0            0.0              0.0     0.0  
           2024-02-27      0.0            0.0              0.0     0.0  
           2024-02-29      0.0            0.0              0.0     0.0  

[3661434 rows x 11 columns]

In [44]:
login_report[login_report['account_currency']=='AUD']

country account_currency    reg_date  \
login    record_time                                           
804687   2023-01-10   Australia              AUD  1673346360   
         2023-01-11   Australia              AUD  1673346360   
         2023-01-12   Australia              AUD  1673346360   
         2023-01-13   Australia              AUD  1673346360   
         2023-01-14   Australia              AUD  1673346360   
...                         ...              ...         ...   
88944965 2024-02-24   Australia              AUD  1673503682   
         2024-02-25   Australia              AUD  1673503682   
         2024-02-26   Australia              AUD  1673503682   
         2024-02-27   Australia              AUD  1673503682   
         2024-02-29   Australia              AUD  1673503682   

                                Datetime  net_deposit  balance   equity  \
login    record_time                                                      
804687   2023-01-10  2023-01-10 10:26:00          0.0     0.00     0.00   
         2023-01-11  2023-01-10 10:26:00          0.0     0.00     0.00   
         2023-01-12  2023-01-10 10:26:00          0.0     0.00     0.00   
         2023-01-13  2023-01-10 10:26:00          0.0     0.00     0.00   
         2023-01-14  2023-01-10 10:26:00          0.0     0.00     0.00   
...                                  ...          ...      ...      ...   
88944965 2024-02-24  2023-01-12 06:08:02          0.0  4881.53  4881.53   
         2024-02-25  2023-01-12 06:08:02          0.0  4881.53  4881.53   
         2024-02-26  2023-01-12 06:08:02          0.0  4956.92  4956.92   
         2024-02-27  2023-01-12 06:08:02          0.0  4993.72  4993.72   
         2024-02-29  2023-01-12 06:08:02          0.0  5106.84  5106.84   

                      credit  profit_closed  profit_floating  margin  
login    record_time                                                  
804687   2023-01-10      0.0           0.00              0.0     0.0  
         2023-01-11      0.0           0.00              0.0     0.0  
         2023-01-12      0.0           0.00              0.0     0.0  
         2023-01-13      0.0           0.00              0.0     0.0  
         2023-01-14      0.0           0.00              0.0     0.0  
...                      ...            ...              ...     ...  
88944965 2024-02-24      0.0           0.00              0.0     0.0  
         2024-02-25      0.0           0.00              0.0     0.0  
         2024-02-26      0.0          75.39              0.0     0.0  
         2024-02-27      0.0          36.80              0.0     0.0  
         2024-02-29      0.0         113.12              0.0     0.0  

[72710 rows x 11 columns]

In [45]:
login_report_USD[login_report_USD['account_currency']=='AUD']

country account_currency    reg_date  \
login    record_time                                           
804687   2023-01-10   Australia              AUD  1673346360   
         2023-01-11   Australia              AUD  1673346360   
         2023-01-12   Australia              AUD  1673346360   
         2023-01-13   Australia              AUD  1673346360   
         2023-01-14   Australia              AUD  1673346360   
...                         ...              ...         ...   
88944965 2024-02-24   Australia              AUD  1673503682   
         2024-02-25   Australia              AUD  1673503682   
         2024-02-26   Australia              AUD  1673503682   
         2024-02-27   Australia              AUD  1673503682   
         2024-02-29   Australia              AUD  1673503682   

                                Datetime  net_deposit       balance  \
login    record_time                                                  
804687   2023-01-10  2023-01-10 10:26:00          0.0  0.000000e+00   
         2023-01-11  2023-01-10 10:26:00          0.0  0.000000e+00   
         2023-01-12  2023-01-10 10:26:00          0.0  0.000000e+00   
         2023-01-13  2023-01-10 10:26:00          0.0  0.000000e+00   
         2023-01-14  2023-01-10 10:26:00          0.0  0.000000e+00   
...                                  ...          ...           ...   
88944965 2024-02-24  2023-01-12 06:08:02          0.0  2.382934e+07   
         2024-02-25  2023-01-12 06:08:02          0.0  2.382934e+07   
         2024-02-26  2023-01-12 06:08:02          0.0  2.457106e+07   
         2024-02-27  2023-01-12 06:08:02          0.0  2.493724e+07   
         2024-02-29  2023-01-12 06:08:02          0.0  2.607981e+07   

                            equity  credit  profit_closed  profit_floating  \
login    record_time                                                         
804687   2023-01-10   0.000000e+00     0.0         0.0000              0.0   
         2023-01-11   0.000000e+00     0.0         0.0000              0.0   
         2023-01-12   0.000000e+00     0.0         0.0000              0.0   
         2023-01-13   0.000000e+00     0.0         0.0000              0.0   
         2023-01-14   0.000000e+00     0.0         0.0000              0.0   
...                            ...     ...            ...              ...   
88944965 2024-02-24   2.382934e+07     0.0         0.0000              0.0   
         2024-02-25   2.382934e+07     0.0         0.0000              0.0   
         2024-02-26   2.457106e+07     0.0      5683.6521              0.0   
         2024-02-27   2.493724e+07     0.0      1354.2400              0.0   
         2024-02-29   2.607981e+07     0.0     12796.1344              0.0   

                      margin  
login    record_time          
804687   2023-01-10      0.0  
         2023-01-11      0.0  
         2023-01-12      0.0  
         2023-01-13      0.0  
         2023-01-14      0.0  
...                      ...  
88944965 2024-02-24      0.0  
         2024-02-25      0.0  
         2024-02-26      0.0  
         2024-02-27      0.0  
         2024-02-29      0.0  

[72710 rows x 11 columns]

In [46]:
login_report_reset = login_report.reset_index()
last_reports = login_report_reset.groupby('login').last()
last_reports[last_reports.index.isin([804687, 804708, 88944936, 88945038, 760487])]

,record_time,country,account_currency,reg_date,Datetime,net_deposit,balance,equity,credit,profit_closed,profit_floating,margin
login,,,,,,,,,,,,
760487,2024-02-29,Singapore,SGD,1672822104,2023-01-04 08:48:24,0.0,96.81,96.81,0.0,0.0,0.0,0.0
804687,2024-02-29,Australia,AUD,1673346360,2023-01-10 10:26:00,0.0,505.03,505.03,0.0,0.0,0.0,0.0
804708,2024-02-29,Malta,EUR,1673522688,2023-01-12 11:24:48,0.0,0.71,0.71,0.0,0.0,0.0,0.0
88944936,2024-02-29,Canada,CAD,1673476264,2023-01-11 22:31:04,0.0,100.00,100.00,0.0,0.0,0.0,0.0
88945038,2024-02-29,Singapore,SGD,1683285920,2023-05-05 11:25:20,0.0,15567.53,15567.53,0.0,0.0,0.0,0.0


In [47]:
login_report_USD_reset = login_report_USD.reset_index()
last_reports_USD = login_report_USD_reset.groupby('login').last()
last_reports_USD[last_reports_USD.index.isin([804687, 804708, 88944936, 88945038, 760487])]

,record_time,country,account_currency,reg_date,Datetime,net_deposit,balance,equity,credit,profit_closed,profit_floating,margin
login,,,,,,,,,,,,
760487,2024-02-29,Singapore,SGD,1672822104,2023-01-04 08:48:24,0.0,9.372176e+03,9.372176e+03,0.0,0.0,0.0,0.0
804687,2024-02-29,Australia,AUD,1673346360,2023-01-10 10:26:00,0.0,2.550553e+05,2.550553e+05,0.0,0.0,0.0,0.0
804708,2024-02-29,Malta,EUR,1673522688,2023-01-12 11:24:48,0.0,5.041000e-01,5.041000e-01,0.0,0.0,0.0,0.0
88944936,2024-02-29,Canada,CAD,1673476264,2023-01-11 22:31:04,0.0,1.000000e+04,1.000000e+04,0.0,0.0,0.0,0.0
88945038,2024-02-29,Singapore,SGD,1683285920,2023-05-05 11:25:20,0.0,2.423480e+08,2.423480e+08,0.0,0.0,0.0,0.0


In [48]:
sorted_last_reports = last_reports.sort_values('balance')
sorted_last_reports

,record_time,country,account_currency,reg_date,Datetime,net_deposit,balance,equity,credit,profit_closed,profit_floating,margin
login,,,,,,,,,,,,
808807,2023-11-06,United Kingdom,GBP,1679005559,2023-03-16 22:25:59,0.00,-4998.72,1.28,5000.0,0.00,0.00,0.00
808806,2023-11-06,United Kingdom,GBP,1679005529,2023-03-16 22:25:29,0.00,-4998.50,1.50,5000.0,0.00,0.00,0.00
7061821,2024-02-29,United Kingdom,USD,1702873503,2023-12-18 04:25:03,0.00,-3535.58,-3535.58,0.0,0.00,0.00,0.00
7057098,2024-02-29,India,USD,1699867921,2023-11-13 09:32:01,0.00,-2714.30,280.70,2995.0,-726.50,0.00,0.00
7058652,2023-12-24,Turkey,USD,1700820464,2023-11-24 10:07:44,0.00,-1981.90,18.10,2000.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
7059314,2024-02-29,Austria,CHF,1701262490,2023-11-29 12:54:50,0.00,375013.79,375013.79,0.0,-1486.31,0.00,0.00
7057066,2024-02-29,United Arab Emirates,USD,1699847589,2023-11-13 03:53:09,-3949.02,388392.14,765523.55,0.0,0.00,377131.41,889389.51
7059315,2024-02-29,Austria,CHF,1701262785,2023-11-29 12:59:45,400000.00,400000.00,400000.00,0.0,0.00,0.00,0.00


In [49]:
login_report[['net_deposit', 'balance', 'equity', 'credit', 'profit_closed', 'profit_floating', 'margin']].groupby('login').mean()

,net_deposit,balance,equity,credit,profit_closed,profit_floating,margin
login,,,,,,,
524972,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
524974,0.704225,350.811056,350.400563,0.0,1.107570,-0.410493,1.378556
524976,-8.732379,2734.360874,2732.854320,0.0,8.732379,-1.506553,3.292767
524978,-148.587567,48864.048289,47623.511749,0.0,148.587567,-1240.536540,805.651749
524979,6.950570,513.283650,205.501445,0.0,-6.950456,-307.782205,49.642015
...,...,...,...,...,...,...,...
1000055123,0.000000,36671.151727,36257.755091,0.0,-22.536364,-413.396636,116.599909
1000055441,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1000055515,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
